[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Igkho/GS-reconstruction/blob/main/train_gs.ipynb)

The first steps of gaussian splatting processing could be done locally.

The images should be processed with colmap (feature extraction, feature matching, reconstruction). Colmap could be installed from https://github.com/colmap/colmap.git. The reconstructed model should be exported onto distorted/sparse directory, i.e. the processed files structure will be as follows:

dataset

>distorted

>> sparse

>>> 0

>>>> camera.bin

>>>> images.bin

>>>> points3D.bin

> input

>> image001.jpg

>> image002.jpg

>> ...


Then the result should be converted (undistorted) with gaussian splatting convert

    git clone https://github.com/nyu-systems/Grendel-GS.git
    cd Grendel-GS
    python convert.py -s {PATH_TO_DATASET} --skip_matching

The processed files structure will be as follows:

dataset

> distorted

> images

>> image001.jpg

>> image002.jpg

>> ...

> input

> sparse

>> 0

>>> camera.bin

>>> images.bin

>>> points3D.bin

The final stage (training of the gaussians) does have the highest requirements for processing hardware. It could be done in Google Colab, connecting to the runtime with GPU:

Mount your Google Drive in Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Check if the selected GPU has CUDA.

In [11]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
    print("CUDA version:", torch.version.cuda)
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")


CUDA is available.
CUDA version: 12.4
GPU: Tesla T4


Clone Grendel Gaussian Splatting repository and submodules (the step could be run only the first time since the data is still stored on your Google Drive)

In [ ]:
%cd /content/drive/MyDrive/
!git clone https://github.com/nyu-systems/Grendel-GS.git
%cd Grendel-GS/submodules
!git clone https://github.com/nyu-systems/diff-gaussian-rasterization.git
!git clone https://github.com/nerfstudio-project/gsplat.git
!git clone https://gitlab.inria.fr/bkerbl/simple-knn.git

Install dependencies and submodules

In [ ]:
%cd /content/drive/MyDrive/Grendel-GS/
!sudo apt install libglm-dev
!pip install plyfile
!pip install submodules/diff-gaussian-rasterization/
!pip install submodules/simple-knn/
!pip install submodules/gsplat/

Download the sample dataset images and reconstructed model and extract them to subfolder (the step could be run only the first time since the data is still stored on your Google Drive)

In [ ]:
from pathlib import Path
Path('/content/drive/MyDrive/dataset').mkdir(parents=True, exist_ok=True)
%cd /content/drive/MyDrive/dataset
!gdown 19tNM74geNq9kbyng5ULfC1H4liH_lhAr
!unzip -o -q data.zip

Set environment variables to maximize GPU memory usage

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
%env PYTORCH_CUDA_ALLOC_CONF

Train the converted dataset.

In [ ]:
%cd /content/drive/MyDrive/Grendel-GS/
!python train.py -s /content/drive/MyDrive/dataset -m /content/drive/MyDrive/dataset/output --iterations 30000 --save_iterations 2000 5000 7000 10000 20000 30000 --test_iterations 0 -w

Download the .ply file from dataset/output/point_cloud/{iteration_num}/point_cloud_rk0_ws1.py

Upload the file to see here: https://playcanvas.com/viewer/